### Bi-Directional LSTM -> POS tagging of ORCHID (Bi-LSTM)

In [0]:
!pip install nltk

In [0]:
import nltk
nltk.download('treebank')
tagged_sentences = nltk.corpus.treebank.tagged_sents()
 
print(tagged_sentences[0])
print("Tagged sentences: ", len(tagged_sentences))
print("Tagged words:", len(nltk.corpus.treebank.tagged_words()))

[nltk_data] Downloading package treebank to /root/nltk_data...
[nltk_data]   Unzipping corpora/treebank.zip.
[('Pierre', 'NNP'), ('Vinken', 'NNP'), (',', ','), ('61', 'CD'), ('years', 'NNS'), ('old', 'JJ'), (',', ','), ('will', 'MD'), ('join', 'VB'), ('the', 'DT'), ('board', 'NN'), ('as', 'IN'), ('a', 'DT'), ('nonexecutive', 'JJ'), ('director', 'NN'), ('Nov.', 'NNP'), ('29', 'CD'), ('.', '.')]
Tagged sentences:  3914
Tagged words: 100676


In [0]:
import numpy as np
 
sentences, sentence_tags =[], [] 
for tagged_sentence in tagged_sentences:
    sentence, tags = zip(*tagged_sentence)
    sentences.append(np.array(sentence))
    sentence_tags.append(np.array(tags))
 
# Let's see how a sequence looks
 
print(sentences[5])
print(sentence_tags[5])

['Lorillard' 'Inc.' ',' 'the' 'unit' 'of' 'New' 'York-based' 'Loews'
 'Corp.' 'that' '*T*-2' 'makes' 'Kent' 'cigarettes' ',' 'stopped' 'using'
 'crocidolite' 'in' 'its' 'Micronite' 'cigarette' 'filters' 'in' '1956'
 '.']
['NNP' 'NNP' ',' 'DT' 'NN' 'IN' 'JJ' 'JJ' 'NNP' 'NNP' 'WDT' '-NONE-' 'VBZ'
 'NNP' 'NNS' ',' 'VBD' 'VBG' 'NN' 'IN' 'PRP$' 'NN' 'NN' 'NNS' 'IN' 'CD'
 '.']


In [0]:
from sklearn.model_selection import train_test_split

(train_sentences, test_sentences, train_tags, test_tags) = train_test_split(sentences, sentence_tags, test_size=0.2)

In [0]:
words, tags = set([]), set([])
 
for s in train_sentences:
    for w in s:
        words.add(w.lower())
 
for ts in train_tags:
    for t in ts:
        tags.add(t)
 
word2index = {w: i + 2 for i, w in enumerate(list(words))}
word2index['-PAD-'] = 0  # The special value used for padding
word2index['-OOV-'] = 1  # The special value used for OOVs
 
tag2index = {t: i + 1 for i, t in enumerate(list(tags))}
tag2index['-PAD-'] = 0  # The special value used to padding

In [0]:
train_sentences_X, test_sentences_X, train_tags_y, test_tags_y = [], [], [], []

In [0]:
for s in train_sentences:
    s_int = []
    for w in s:
        try:
            s_int.append(word2index[w.lower()])
        except KeyError:
            s_int.append(word2index['-OOV-'])
 
    train_sentences_X.append(s_int)

In [0]:
for s in test_sentences:
    s_int = []
    for w in s:
        try:
            s_int.append(word2index[w.lower()])
        except KeyError:
            s_int.append(word2index['-OOV-'])
 
    test_sentences_X.append(s_int)

In [0]:
for s in train_tags:
    train_tags_y.append([tag2index[t] for t in s])

In [0]:
for s in test_tags:
  test_tags_y.append([tag2index[t] for t in s])

In [0]:
print(train_sentences_X[0])
print(test_sentences_X[0])
print(train_tags_y[0])
print(test_tags_y[0])

[532, 2155, 9098, 7123, 9631, 8218, 3074, 8457, 854, 9464, 363, 6566, 7650, 6711, 5649, 9527, 8880, 3643, 6711, 5558, 9527, 8218, 3074, 7125, 854, 5327]
[9824, 5619, 3458, 3395, 5490, 5610, 8880, 8893, 9018, 5251, 2258, 870, 6995, 5610, 8880, 5771, 426, 4778, 6504, 4442, 4778, 6566, 8880, 9205, 5019, 4778, 3806, 7655, 532, 426, 5327]
[32, 44, 21, 43, 38, 42, 5, 5, 18, 18, 21, 38, 40, 4, 38, 38, 32, 40, 4, 4, 38, 42, 5, 5, 18, 19]
[24, 43, 38, 13, 35, 38, 32, 4, 4, 4, 36, 29, 18, 38, 32, 40, 4, 2, 25, 4, 2, 38, 32, 29, 4, 2, 37, 28, 32, 4, 19]


In [0]:
MAX_LENGTH = len(max(train_sentences_X, key=len))
print(MAX_LENGTH)

271


In [0]:
from keras.preprocessing.sequence import pad_sequences

Using TensorFlow backend.


In [0]:
train_sentences_X = pad_sequences(train_sentences_X,
                                  maxlen=MAX_LENGTH, padding='post')
test_sentences_X = pad_sequences(test_sentences_X,
                                 maxlen=MAX_LENGTH, padding='post')
train_tags_y = pad_sequences(train_tags_y,
                             maxlen=MAX_LENGTH, padding='post')
test_tags_y = pad_sequences(test_tags_y,
                            maxlen=MAX_LENGTH, padding='post')

In [0]:
print(train_sentences_X[0])
print(test_sentences_X[0])
print(train_tags_y[0])
print(test_tags_y[0])

[ 532 2155 9098 7123 9631 8218 3074 8457  854 9464  363 6566 7650 6711
 5649 9527 8880 3643 6711 5558 9527 8218 3074 7125  854 5327    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0 

In [0]:
from keras import backend as K
 
def ignore_class_accuracy(to_ignore=0):
    def ignore_accuracy(y_true, y_pred):
        y_true_class = K.argmax(y_true, axis=-1)
        y_pred_class = K.argmax(y_pred, axis=-1)
 
        ignore_mask = K.cast(K.not_equal(y_pred_class, to_ignore), 'int32')
        matches = K.cast(K.equal(y_true_class, y_pred_class), 'int32') * ignore_mask
        accuracy = K.sum(matches) / K.maximum(K.sum(ignore_mask), 1)
        return accuracy
    return ignore_accuracy

In [0]:
from keras.models import Sequential
from keras.layers import Dense, LSTM, InputLayer, Bidirectional, TimeDistributed, Embedding, Activation
from keras.optimizers import Adam

In [0]:
model = Sequential()
model.add(InputLayer(input_shape=(MAX_LENGTH, )))
model.add(Embedding(len(word2index), 128))
model.add(Bidirectional(LSTM(256, return_sequences=True)))
model.add(TimeDistributed(Dense(len(tag2index))))
model.add(Activation('softmax'))

In [0]:
model.compile(loss='categorical_crossentropy',
              optimizer=Adam(0.001),
              metrics=['accuracy', ignore_class_accuracy(0)])

In [0]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 271, 128)          1291776   
_________________________________________________________________
bidirectional_1 (Bidirection (None, 271, 512)          788480    
_________________________________________________________________
time_distributed_1 (TimeDist (None, 271, 47)           24111     
_________________________________________________________________
activation_1 (Activation)    (None, 271, 47)           0         
Total params: 2,104,367
Trainable params: 2,104,367
Non-trainable params: 0
_________________________________________________________________


In [0]:
def to_categorical(sequences, categories):
    cat_sequences = []
    for s in sequences:
        cats = []
        for item in s:
            cats.append(np.zeros(categories))
            cats[-1][item] = 1.0
        cat_sequences.append(cats)
    return np.array(cat_sequences)

In [0]:
cat_train_tags_y = to_categorical(train_tags_y, len(tag2index))
print(cat_train_tags_y[0])

[[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 1. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [1. 0. 0. ... 0. 0. 0.]
 [1. 0. 0. ... 0. 0. 0.]
 [1. 0. 0. ... 0. 0. 0.]]


In [0]:
history = model.fit(train_sentences_X,
                    to_categorical(train_tags_y, len(tag2index)),
                    batch_size=128,
                    epochs=40,
                    validation_split=0.2)

Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where



Train on 2504 samples, validate on 627 samples
Epoch 1/40





2504/2504 [==============================] - 22s 9ms/step - loss: 1.3105 - acc: 0.8584 - ignore_accuracy: 0.0210 - val_loss: 0.3892 - val_acc: 0.9089 - val_ignore_accuracy: 0.1309
Epoch 2/40
2504/2504 [==============================] - 16s 7ms/step - loss: 0.3483 - acc: 0.9046 - ignore_accuracy: 0.0488 - val_loss: 0.3133 - val_acc: 0.9083 - val_ignore_accuracy: 0.0000e+00
Epoch 3/40
2504/2504 [==============================] - 16s 7ms/step - loss: 0.3192 - acc: 0.9080 - ignore_accuracy: 0.0906 - val_loss: 0.3009 - val_acc: 0.9180 - val_ignore_accuracy: 0.1313
Epoch 4/40
2504/2504 [==============================] - 16s 6ms/step - loss: 0.3084 - acc: 0.9155 - ignore_accuracy: 0.1307 - val_loss: 0.2932 - val_acc: 0.9205 - val_ignore_accuracy: 0.1391
Epoch 5/40
2504/2504 [==============================] - 16s 7ms/step - loss: 0.2999

In [0]:
test_samples = [
    "singing is my hobby .".split(),
    "She was studying every week for a year .".split()
]
print(test_samples)

[['singing', 'is', 'my', 'hobby', '.'], ['She', 'was', 'studying', 'every', 'week', 'for', 'a', 'year', '.']]


In [0]:
test_samples_X = []
for s in test_samples:
    s_int = []
    for w in s:
        try:
            s_int.append(word2index[w.lower()])
        except KeyError:
            s_int.append(word2index['-OOV-'])
    test_samples_X.append(s_int)
 
test_samples_X = pad_sequences(test_samples_X, maxlen=MAX_LENGTH, padding='post')
print(test_samples_X)

[[   1 5375 5233    1 5327    0    0    0    0    0    0    0    0    0
     0    0    0    0    0    0    0    0    0    0    0    0    0    0
     0    0    0    0    0    0    0    0    0    0    0    0    0    0
     0    0    0    0    0    0    0    0    0    0    0    0    0    0
     0    0    0    0    0    0    0    0    0    0    0    0    0    0
     0    0    0    0    0    0    0    0    0    0    0    0    0    0
     0    0    0    0    0    0    0    0    0    0    0    0    0    0
     0    0    0    0    0    0    0    0    0    0    0    0    0    0
     0    0    0    0    0    0    0    0    0    0    0    0    0    0
     0    0    0    0    0    0    0    0    0    0    0    0    0    0
     0    0    0    0    0    0    0    0    0    0    0    0    0    0
     0    0    0    0    0    0    0    0    0    0    0    0    0    0
     0    0    0    0    0    0    0    0    0    0    0    0    0    0
     0    0    0    0    0    0    0    0    0    0    0    0   

In [0]:
predictions = model.predict(test_samples_X)
print(predictions, predictions.shape)

[[[7.8720022e-03 1.0827631e-02 2.1617103e-04 ... 2.7352381e-01
   3.6657455e-03 6.2896637e-05]
  [3.3354718e-06 9.1246405e-04 1.0949839e-03 ... 1.8295063e-06
   8.0726357e-05 3.2103391e-04]
  [9.8364406e-05 1.8692419e-03 4.6534743e-04 ... 4.5884450e-04
   1.1623140e-02 3.0030494e-03]
  ...
  [9.9998844e-01 1.0128198e-09 4.2514735e-11 ... 5.0886217e-10
   7.9631135e-10 2.9319154e-09]
  [9.9998283e-01 2.1384057e-09 1.0655595e-10 ... 5.2491417e-10
   1.3060846e-09 8.1244060e-09]
  [9.9997294e-01 4.4311017e-09 2.4459237e-10 ... 5.8346777e-10
   2.1529294e-09 2.0349503e-08]]

 [[8.0319005e-06 1.3802830e-02 1.5102000e-03 ... 2.3775834e-03
   8.1847963e-04 5.4658347e-05]
  [7.1564011e-07 1.7370489e-04 9.6323283e-06 ... 4.4972057e-07
   5.8691170e-05 6.1362043e-05]
  [1.8270841e-04 7.2281633e-05 9.5876248e-06 ... 7.3228934e-04
   1.9747106e-04 6.4861804e-04]
  ...
  [9.9998844e-01 1.0128141e-09 4.2514655e-11 ... 5.0886217e-10
   7.9630524e-10 2.9318932e-09]
  [9.9998283e-01 2.1383895e-09 1.065

In [0]:
def logits_to_tokens(sequences, index):
    token_sequences = []
    for categorical_sequence in sequences:
        token_sequence = []
        for categorical in categorical_sequence:
            token_sequence.append(index[np.argmax(categorical)])
 
        token_sequences.append(token_sequence)
 
    return token_sequences

In [0]:
predictions = model.predict(test_samples_X)
tokens = logits_to_tokens(predictions, {i: t for t, i in tag2index.items()})
for i in tokens:
  print(i)

['NNP', 'VBZ', 'PRP$', 'NN', '.', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-P

In [0]:
#singing is my hobby.
#She was studying every week for a year.